In [2]:
from torchvision.datasets import FashionMNIST

In [115]:
from torch.utils.data import DataLoader, TensorDataset
train = FashionMNIST('.', download=True, train=True)
dl = DataLoader(TensorDataset(train.train_data, train.train_labels), batch_size=int(len(train) * 0.3))
it = iter(dl)
valid = next(it)
valid_dl = DataLoader(TensorDataset(valid[0].type(torch.float32), valid[1]), batch_size=30)
Xs = []
ys = []
for i in it:
    Xs.append(i[0].type(torch.float32))
    ys.append(i[1])

train_X = torch.cat(Xs, 0)
    
train_dl = DataLoader(TensorDataset(train_X, torch.cat(ys, 0)), batch_size=30)
# valid[0].size(), valid[1].size()


In [116]:
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F


class FirstCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

def loss_batch(model, loss_func, xb, yb, opt=None):
#     print(model(xb).type(), yb.type())
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)


model = FirstCNN()
fit(20, model, F.cross_entropy, torch.optim.SGD(model.parameters(), lr=0.1), train_dl, valid_dl)
print("DON!")

0 1.0546546453237533
1 0.7579363002131383
2 0.6592250390102466
3 0.6393157167732716
4 0.5863012673705816
5 0.5547775803258022
6 0.5493090981493394
7 0.5114225192616383
8 0.5704641215999922
9 0.6606783599158128
10 0.5425671192879478
11 0.5549322261288762
12 0.5536189061279099
13 0.5563876662403345
14 0.54101467885077
15 0.5277495332310597
16 0.5394995466123025
17 0.5303962248315414
18 0.53318738874048
19 0.5385533541192611
DON!


In [64]:
import numpy as np
from matplotlib.pyplot import imshow
#imshow(np.asarray(train[7][0]))
# train[7]
from torch.utils.data import TensorDataset
# train_ds = TensorDataset(train.train_data, train.train_labels
# TensorDataset(train)[:30000]
[method_name for method_name in dir(train) if callable(getattr(train, method_name))]
# len(train)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_check_exists',
 'download']

In [37]:
import torch
from skorch import NeuralNetClassifier
net_regr = NeuralNetClassifier(
    module=FirstCNN,
    max_epochs=20,
    lr=0.1,
    criterion=torch.nn.NLLLoss,
#     criterion__weight=weight,
    optimizer=torch.optim.SGD,
    optimizer__momentum=0.9,
#     device='cuda',  # uncomment this to train with CUDA
)

In [30]:
train.train_labels.unique()

tensor([1, 9, 0, 3, 2, 7, 5, 6, 4, 8])

In [34]:
train.train_data = train.train_data.type('torch.FloatTensor')
net_regr.fit(train.train_data, train.train_labels)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1           inf       0.1000      -16.1242  15.5629
      2      -16.6445       0.1000      -17.0698  15.5948
      3      -17.3730       0.1004      -17.6433  15.3967
      4      -17.8578       0.1019      -18.0565  15.9598
      5      -18.2226       0.1047      -18.3799  15.4213
      6      -18.5153       0.1094      -18.6457  15.9111
      7      -18.7600       0.1146      -18.8713  15.4875
      8      -18.9701       0.1199      -19.0674  15.7156
      9      -19.1544       0.1279      -19.2407  15.3496
     10      -19.3184       0.1368      -19.3960  15.3957
     11      -19.4662       0.1431      -19.5367  15.3407
     12      -19.6007       0.1486      -19.6653  15.9689
     13      -19.7241       0.1552      -19.7838  15.3812
     14      -19.8381       0.1616      -19.8936  15.3919
     15      -19.9441       0.1651      -19.9959  15.3365
     16      -

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=FirstCNN(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(16, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  ),
)

In [117]:
test.test_data = test.test_data.type('torch.FloatTensor')
preds = model(test.test_data)

In [119]:
preds

tensor([[ 4.0686,  0.9486,  1.1147,  ..., 16.0540,  7.7589, 17.8792],
        [ 4.8934,  3.0425, 10.1708,  ...,  0.6244,  2.9623,  1.0336],
        [ 8.1671, 18.2464,  6.7151,  ...,  1.0052,  4.3644,  1.4280],
        ...,
        [ 4.0026,  4.2633,  5.2982,  ...,  2.3113, 12.9487,  3.0310],
        [ 6.9058, 19.4025,  3.5864,  ...,  1.3586,  6.0631,  3.3358],
        [ 1.8625,  0.0587,  2.2747,  ..., 11.4606,  9.0390,  6.7349]],
       grad_fn=<ViewBackward>)

In [118]:
import pandas as pd
df = pd.DataFrame()
df['Class'] = preds
df.index.name = 'Id'
df.to_csv('submission1.csv')
df

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series